In [1]:
import json
import codecs
from datetime import datetime
import numpy as np

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve

import pydotplus
from IPython.display import Image 


In [2]:
path= "english10k_tweets.json"

tweets = []
with open(path) as f:
    for line in f:
        tweets.append(json.loads(line))

In [3]:
len(tweets)

10000

In [4]:
device_type= {'Twitter for Iphone':'mobile', 'Twitter for Android':'mobile',
    'Twitter Web Client':'web', "Tweetbot for Mac":'web',
    'Instagram':'app', 'Tumblr':'app', 'Foursquared':'app',
    'Falcon Social Media Management':'smm', 'TweetDeck':'smm', 'dlvr.it':'smm',
    'Trendsmap Alerting':'bot', 'SpotifyNowPlaying':'bot'}

In [5]:
target_name=list(device_type.values())
target_name.append('other')
target_name=list(set(target_name))
device_type2ID=dict(zip(target_name,range(len(target_name))))
ID2device_type=dict(zip(range(len(target_name)),target_name))


In [6]:
tweet_date_created={}
count_favourites_day={}
user_date_created={}
c=0

for tweet in tweets:
    d_tweet=tweet['created_at']
    d_user=tweet['created_at']
    datetime_tweet = datetime.strptime(d_tweet[4:-20] + d_tweet[-5:], '%b %d %Y')
    datetime_user = datetime.strptime(d_user[4:-20] + d_user[-5:], '%b %d %Y')
    if tweet['user']['id'] in tweet_date_created.keys():
        tweet_date_created[tweet['user']['id']].append(datetime_tweet)
        count_favourites_day[tweet['user']['id']].append(tweet['user']['favourites_count'])
    else:
        tweet_date_created[tweet['user']['id']]= [datetime_tweet]
        user_date_created[tweet['user']['id']] = datetime_user
        count_favourites_day[tweet['user']['id']]= [tweet['user']['favourites_count']]
        


In [7]:
properties_tweet=[]
target= []
for tweet in tweets:
    len_text = len(tweet['text'].split(' '))
    target.append(tweet['id'])
    properties_tweet.append([tweet['in_reply_to_user_id']])
    properties_tweet[-1].append(str(tweet['retweeted']))
    try:
        properties_tweet[-1].append(str(tweet['user']['followers_count']/(tweet['user']['followers_count']+tweet['user']['friends_count'])))
    except:
        properties_tweet[-1].append('0')
    properties_tweet[-1].append(len(tweet['entities']['hashtags'])/ len_text)
    properties_tweet[-1].append(len(tweet['entities']['urls'])/ len_text)
    properties_tweet[-1].append(len(tweet['entities']['user_mentions'])/ len_text)
    properties_tweet[-1].append(len(tweet_date_created[tweet['user']['id']]) / 
                               ((tweet_date_created[tweet['user']['id']][-1] - user_date_created[tweet['user']['id']]).days+1))
    properties_tweet[-1].append(tweet['user']['favourites_count'] / 
                              ((tweet_date_created[tweet['user']['id']][-1] - user_date_created[tweet['user']['id']]).days+1))
#     count(w['user']['id']) / (date now or date last tweet) - count(tweet['user']['created_at'])
#     count(w['user']['favourites_count']) / (date now or date last tweet) - count(tweet['user']['created_at'])
    try:
        properties_tweet[-1].append(device_type[tweet['source'].strip().split('>')[-2].split('<')[0]])
    except:
        properties_tweet[-1].append('other')


In [8]:
properties_tweet= np.array(properties_tweet)
properties_tweet.shape

(10000, 9)